In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd


In [2]:
import pandas as pd

# Load the dataset
file_path = "merged_telemetry_and_pitstop.csv"
data = pd.read_csv(file_path)



   Unnamed: 0 Grand Prix  Race                       Circuit   Year  \
0           0    Bahrain     3  Bahrain International Circuit  2014   
1           1    Bahrain     3  Bahrain International Circuit  2014   
2           2    Bahrain     3  Bahrain International Circuit  2014   
3           3    Bahrain     3  Bahrain International Circuit  2014   
4           4    Bahrain     3  Bahrain International Circuit  2014   

           Driver_x  Stop   Lap Stop    Time_x Duration  ... Unnamed: 11  \
0  Jean-Eric Vergne      1         1  18:06:44   33.895  ...         NaN   
1  Jean-Eric Vergne      1         1  18:06:44   33.895  ...         NaN   
2  Jean-Eric Vergne      1         1  18:06:44   33.895  ...         NaN   
3  Jean-Eric Vergne      1         1  18:06:44   33.895  ...         NaN   
4  Jean-Eric Vergne      1         1  18:06:44   33.895  ...         NaN   

   Speed  Throttle  Brake  nGear  DRS                  Time_y  Driver_y  \
0    245        84  False      6    1  0 

/var/folders/7q/w_k7x3ss7fsgflp_442s596r0000gn/T/ipykernel_4519/849844411.py:5: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Load the dataset
file_path = "merged_telemetry_and_pitstop.csv"
data = pd.read_csv(file_path)

# Drop rows with missing values in relevant columns
features = ['Speed', 'Throttle', 'Brake', 'nGear', 'DRS', 'RaceName']  # Include categorical features here
target = 'Winner'

data = data.dropna(subset=features + [target])

# Split data into features (X) and target (y)
X = data[features]
y = pd.get_dummies(data[target], drop_first=True)  # Binary encoding for the target variable

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing for numeric and categorical features
numeric_features = ['Speed', 'Throttle', 'Brake', 'nGear', 'DRS']
categorical_features = ['RaceName']

# Preprocessing pipelines
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Build the model
# Build the model
model = Sequential([
    Dense(64, input_dim=X_train_processed.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Use softmax for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_processed, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_processed, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Predict
predictions = model.predict(X_test_processed)
print("Predictions:", predictions)


/var/folders/7q/w_k7x3ss7fsgflp_442s596r0000gn/T/ipykernel_4519/3405700013.py:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8017/8017 ━━━━━━━━━━━━━━━━━━━━ 4s 426us/step - accuracy: 0.4897 - loss: 3.0789 - val_accuracy: 0.5530 - val_loss: 15.6472
Epoch 2/50
8017/8017 ━━━━━━━━━━━━━━━━━━━━ 3s 422us/step - accuracy: 0.4046 - loss: 30.4012 - val_accuracy: 0.5280 - val_loss: 96.6071
Epoch 3/50
8017/8017 ━━━━━━━━━━━━━━━━━━━━ 3s 416us/step - accuracy: 0.3978 - loss: 90.3690 - val_accuracy: 0.1802 - val_loss: 85.5724
Epoch 4/50
8017/8017 ━━━━━━━━━━━━━━━━━━━━ 3s 417us/step - accuracy: 0.3979 - loss: 189.1370 - val_accuracy: 0.2336 - val_loss: 216.4244
Epoch 5/50
8017/8017 ━━━━━━━━━━━━━━━━━━━━ 3s 418us/step - accuracy: 0.3992 - loss: 323.6795 - val_accuracy: 0.3403 - val_loss: 200.9570
Epoch 6/50
8017/8017 ━━━━━━━━━━━━━━━━━━━━ 3s 415us/step - accuracy: 0.3975 - loss: 488.8365 - val_accuracy: 0.3364 - val_loss: 465.1449
Epoch 7/50
8017/8017 ━━━━━━━━━━━━━━━━━━━━ 3s 418us/step - accuracy: 0.3949 - loss: 672.2428 - val_accuracy: 0.5293 - val_loss: 595.0939
Epoch 8/50
8017/8017 ━━━━━━━━━━━━━━━━━━━━ 3s 416us/step - accuracy

In [6]:
import numpy as np

class_distribution = np.sum(y_train, axis=0)
print("Class Distribution in Training Set:", class_distribution)


Class Distribution in Training Set: Lewis Hamilton      135473
Max Verstappen       99608
Nico Rosberg         31699
Sebastian Vettel     22144
Sergio Perez          6105
Valtteri Bottas       6394
dtype: int64


In [8]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_processed, y_train)

print("Balanced Class Distribution:", y_train_balanced.sum(axis=0))


AttributeError: 'DataFrame' object has no attribute 'argmax'

In [9]:
import numpy as np

# Convert one-hot encoded labels to a single column of class indices
y_train_single = np.argmax(y_train.values, axis=1)


In [10]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_processed, y_train_single)

In [11]:
from tensorflow.keras.utils import to_categorical

y_train_balanced_onehot = to_categorical(y_train_balanced)


In [12]:
model.fit(
    X_train_balanced, y_train_balanced_onehot,
    epochs=50, batch_size=32,
    validation_split=0.2, verbose=1
)


Epoch 1/50
23207/23207 ━━━━━━━━━━━━━━━━━━━━ 10s 413us/step - accuracy: 0.3358 - loss: 70831.1250 - val_accuracy: 0.2011 - val_loss: 754743.0000
Epoch 2/50
23207/23207 ━━━━━━━━━━━━━━━━━━━━ 10s 415us/step - accuracy: 0.3857 - loss: 63690.6836 - val_accuracy: 0.2011 - val_loss: 818853.6875
Epoch 3/50
23207/23207 ━━━━━━━━━━━━━━━━━━━━ 10s 412us/step - accuracy: 0.4023 - loss: 60857.1172 - val_accuracy: 0.2011 - val_loss: 625712.3750
Epoch 4/50
23207/23207 ━━━━━━━━━━━━━━━━━━━━ 10s 409us/step - accuracy: 0.4114 - loss: 57538.0859 - val_accuracy: 0.2010 - val_loss: 641267.6250
Epoch 5/50
23207/23207 ━━━━━━━━━━━━━━━━━━━━ 10s 416us/step - accuracy: 0.4137 - loss: 54416.4141 - val_accuracy: 0.1814 - val_loss: 903281.5000
Epoch 6/50
23207/23207 ━━━━━━━━━━━━━━━━━━━━ 10s 414us/step - accuracy: 0.4155 - loss: 52263.2617 - val_accuracy: 0.2011 - val_loss: 73498.4922
Epoch 7/50
23207/23207 ━━━━━━━━━━━━━━━━━━━━ 9s 408us/step - accuracy: 0.4185 - loss: 50205.2031 - val_accuracy: 0.2011 - val_loss: 51498.

In [17]:
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

# One-hot encode the balanced training labels
y_train_balanced_onehot = onehot_encoder.fit_transform(y_train_balanced.reshape(-1, 1))

# Convert y_test to a NumPy array and reshape it for one-hot encoding
y_test_array = y_test.to_numpy().reshape(-1, 1)
y_test_onehot = onehot_encoder.transform(y_test_array)


In [31]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Ensure scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test_processed)

# Ensure y_train_balanced and y_test are one-hot encoded
# Ensure y_train_balanced and y_test are one-hot encoded
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train_balanced_onehot = onehot_encoder.fit_transform(y_train_balanced.reshape(-1, 1))  # Reshape directly
y_test_onehot = onehot_encoder.transform(y_test.to_numpy().reshape(-1, 1))  # Convert to numpy and reshape


# Debugging shapes
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train_balanced_onehot shape: {y_train_balanced_onehot.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"y_test_onehot shape: {y_test_onehot.shape}")

# Adjust model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(6, activation='softmax')  # For multi-class classification
])

# Compile with categorical crossentropy for multi-class problems
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train_scaled, y_train_balanced_onehot,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# Evaluate on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test_onehot, verbose=1)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


X_train_scaled shape: (928242, 10)
y_train_balanced_onehot shape: (928242, 6)
X_test_scaled shape: (80165, 10)
y_test_onehot shape: (480990, 6)
Epoch 1/50


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 571us/step - accuracy: 0.5246 - loss: 1.0224 - val_accuracy: 0.2011 - val_loss: 2.3434
Epoch 2/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 611us/step - accuracy: 0.5480 - loss: 0.9804 - val_accuracy: 0.2011 - val_loss: 2.5007
Epoch 3/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 6s 538us/step - accuracy: 0.5484 - loss: 0.9796 - val_accuracy: 0.2011 - val_loss: 2.4839
Epoch 4/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 6s 523us/step - accuracy: 0.5489 - loss: 0.9805 - val_accuracy: 0.2011 - val_loss: 2.5681
Epoch 5/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 6s 532us/step - accuracy: 0.5491 - loss: 0.9788 - val_accuracy: 0.2011 - val_loss: 2.5047
Epoch 6/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 6s 530us/step - accuracy: 0.5495 - loss: 0.9788 - val_accuracy: 0.2011 - val_loss: 2.6053
Epoch 7/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 6s 552us/step - accuracy: 0.5488 - loss: 0.9794 - val_accuracy: 0.2011 - val_loss: 2.5635
Epoch 8/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 6s 551us/step - accuracy: 0.5

KeyboardInterrupt: 

In [26]:
y_test_onehot = onehot_encoder.fit_transform(y_test.to_numpy().reshape(-1, 1))
print(f"Corrected y_test_onehot shape: {y_test_onehot.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"y_test_onehot shape: {y_test_onehot.shape}")



Corrected y_test_onehot shape: (480990, 2)
X_test_scaled shape: (80165, 10)
y_test_onehot shape: (480990, 2)


In [38]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Ensure y_test is a Numpy array
y_test = np.array(y_test)  # Convert to Numpy array if not already

# One-hot encode y_test
onehot_encoder = OneHotEncoder(sparse_output=False)
y_test_onehot = onehot_encoder.fit_transform(y_test.reshape(-1, 1))  # Reshape for encoding

# Align shapes of y_test_onehot and X_test_scaled
if y_test_onehot.shape[0] != X_test_scaled.shape[0]:
    y_test_onehot = y_test_onehot[:X_test_scaled.shape[0]]  # Truncate to match
    print(f"Aligned y_test_onehot shape: {y_test_onehot.shape}")

print(f"Final y_test_onehot shape: {y_test_onehot.shape}")


Aligned y_test_onehot shape: (80165, 2)
Final y_test_onehot shape: (80165, 2)


In [39]:
y_test = np.array(y_test)


In [40]:
print(f"Aligned y_test_onehot shape: {y_test_onehot.shape}")
print(f"Final y_test_onehot shape: {y_test_onehot.shape}")


Aligned y_test_onehot shape: (80165, 2)
Final y_test_onehot shape: (80165, 2)


In [41]:
# Evaluate on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test_onehot, verbose=1)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


2024-12-05 18:44:12.137017: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: logits and labels must be broadcastable: logits_size=[32,6] labels_size=[32,2]
	 [[{{function_node __inference_one_step_on_data_15722894}}{{node compile_loss/categorical_crossentropy/softmax_cross_entropy_with_logits}}]]


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/categorical_crossentropy/softmax_cross_entropy_with_logits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/kirtan_patel/Library/Python/3.11/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/kirtan_patel/Library/Python/3.11/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/kirtan_patel/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/kirtan_patel/Library/Python/3.11/lib/python/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", 

In [37]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# Ensure scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_balanced)
X_test_scaled = scaler.transform(X_test_processed)

# Ensure y_train_balanced and y_test are one-hot encoded
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train_balanced_onehot = onehot_encoder.fit_transform(np.array(y_train_balanced).reshape(-1, 1))  # Convert to array and reshape
y_test_onehot = onehot_encoder.transform(np.array(y_test).reshape(-1, 1))  # Convert to array and reshape

# Debugging shapes
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train_balanced_onehot shape: {y_train_balanced_onehot.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print(f"y_test_onehot shape: {y_test_onehot.shape}")

# Adjust model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(y_train_balanced_onehot.shape[1], activation='softmax')  # Automatically match output dimensions
])

# Compile with categorical crossentropy for multi-class problems
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train_scaled, y_train_balanced_onehot,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

# Evaluate on the test set
loss, accuracy = model.evaluate(X_test_scaled, y_test_onehot, verbose=1)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


X_train_scaled shape: (928242, 10)
y_train_balanced_onehot shape: (928242, 6)
X_test_scaled shape: (80165, 10)
y_test_onehot shape: (480990, 6)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 542us/step - accuracy: 0.5264 - loss: 1.0174 - val_accuracy: 0.2011 - val_loss: 2.3175
Epoch 2/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 6s 541us/step - accuracy: 0.5479 - loss: 0.9817 - val_accuracy: 0.2011 - val_loss: 2.5237
Epoch 3/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 574us/step - accuracy: 0.5480 - loss: 0.9799 - val_accuracy: 0.2011 - val_loss: 2.5247
Epoch 4/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 570us/step - accuracy: 0.5494 - loss: 0.9786 - val_accuracy: 0.2011 - val_loss: 2.5450
Epoch 5/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 577us/step - accuracy: 0.5487 - loss: 0.9789 - val_accuracy: 0.2011 - val_loss: 2.4383
Epoch 6/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 560us/step - accuracy: 0.5505 - loss: 0.9776 - val_accuracy: 0.2011 - val_loss: 2.5192
Epoch 7/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 570us/step - accuracy: 0.5502 - loss: 0.9782 - val_accuracy: 0.2011 - val_loss: 2.5017
Epoch 8/50
11604/11604 ━━━━━━━━━━━━━━━━━━━━ 7s 574us/step - ac

KeyboardInterrupt: 